In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")
import snowflake.connector

In [2]:
#  set current working directory
path ='/Users/richakumari/Desktop/Middle East scheduling model -testing codes/inputs_processing/'
import os
os.chdir(path)
os.getcwd()


'/Users/richakumari/Desktop/Middle East scheduling model -testing codes/inputs_processing'

In [3]:
# start_date = input("Enter the start date (YYYY-MM-DD): ")
# end_date = input("Enter the end date (YYYY-MM-DD): ")
# fleet_size = input("Enter the fleet_size: ")

query="""with contracts_base as
(
select
CASE WHEN RIDER_OPS_CODES_CLEAN = 'Albarq Co_FL' THEN 'T2' ELSE 'T1' END AS kuwait_tier,
ZONE_CODE,
Date (STARTS_AT_LOCAL) as start_date,
CAST(EXTRACT(HOUR FROM TO_TIMESTAMP(micro_contracts.starts_at_local )) AS INT) AS starts_at_local_hour_of_day,
sum(CONTRACT_DURATION_WORKED_MINS)/sum(CONTRACT_DURATION_MINS) as contract_duration_worked_prct,
count(distinct RIDER_ID) as rider_count
from
PRODUCTION.DENORMALISED.MICRO_CONTRACTS
where
trim(upper(COUNTRY_NAME)) ='KUWAIT'
and IS_CANCELLED =FALSE
and RIDER_OPS_CODES_CLEAN = 'Albarq Co_FL'
and date (starts_at_local) between '2024-09-01' and '2024-09-07'
group by 1,2,3,4
order by 3

)
,date_shift_contracts as
(
select
*
, case when starts_at_local_hour_of_day in (0,1,2,3,4,5,6) then (start_date -1) else start_date end as start_date_updated
-- , DAYNAME(TO_DATE(start_date_updated))as WEEKDAY
, DAYOFWEEK(TO_DATE(start_date_updated))+1 as WEEKDAY
from contracts_base
)
, avg_rider_attendance as
(
select
zone_code
,case when starts_at_local_hour_of_day =0 then 24
when starts_at_local_hour_of_day =1 then 25
when starts_at_local_hour_of_day =2 then 26
when starts_at_local_hour_of_day =3 then 27
when starts_at_local_hour_of_day =4 then 28
when starts_at_local_hour_of_day =5 then 29
when starts_at_local_hour_of_day =6 then 30
else starts_at_local_hour_of_day end as starts_at_local_hour_of_day
,WEEKDAY
, avg (coalesce (contract_duration_worked_prct, 0)) as avg_contract_duration_worked_prct
, avg (coalesce(rider_count,0))as avg_rider_count
, avg (coalesce(contract_duration_worked_prct*rider_count,0))as attended_riders
from date_shift_contracts
group by 1,2,3
)
, OV_base as
(
select
distinct
zone_code
,(CAST(EXTRACT(HOUR FROM TO_TIMESTAMP_NTZ(coalesce(LOCAL_TIME_TARGET_READY_AT, LOCAL_TIME_DELIVERED_AT,LOCAL_TIME_CREATED_AT) )) AS INT)) AS hour_of_day
,ORDER_DATE
,ID
from PRODUCTION.denormalised.ORDERS
where
upper(trim(country_name)) ='KUWAIT'
and upper(trim(city_name))='KUWAIT'
and FULFILLMENT_TYPE ='Deliveroo'
and status ='DELIVERED'
and order_date between '2024-09-01' and '2024-09-07'
and cluster_name not in ('ASA-CAJ-KAJ-KAK-KAZ-KBN-KDY-SAA-WAF')
)
,date_shift_OV as
(
select
zone_code
,case when hour_of_day in (0,1,2,3,4,5,6) then (ORDER_DATE -1) else ORDER_DATE end as ORDER_DATE_UPDATED
-- ,DAYNAME(TO_DATE(ORDER_DATE_UPDATED))as WEEKDAY
, DAYOFWEEK(TO_DATE(ORDER_DATE_UPDATED))+1 as WEEKDAY
,hour_of_day
,count(distinct ID)as OV
from OV_base
group by 1,2,3,4
)
, OV_final as(
select
zone_code
,case when hour_of_day =0 then 24
when hour_of_day =1 then 25
when hour_of_day =2 then 26
when hour_of_day =3 then 27
when hour_of_day =4 then 28
when hour_of_day =5 then 29
when hour_of_day =6 then 30
else hour_of_day end as hour_of_day_updated
,WEEKDAY
,avg(OV)as avg_OV
,sum(OV)as total_OV
from date_shift_OV
group by 1,2,3
)
,master_join as
(
select
distinct
a.zone_code
,b.hour_of_day_updated
,c.weekday
from OV_base as a
left join
(select distinct hour_of_day_updated from OV_final) as b
on 1=1
left join
(select distinct weekday  from OV_final) as c
on 1=1
)
, final_OP as(
select
base.zone_code
,base.hour_of_day_updated
,base.weekday
,coalesce(OV.total_OV, 0) as total_OV
,coalesce(attend.AVG_CONTRACT_DURATION_WORKED_PRCT,0) as AVG_CONTRACT_DURATION_WORKED_PRCT
from master_join as base
left join
OV_final as OV
on base.zone_code=OV.zone_code
and base.hour_of_day_updated=OV.hour_of_day_updated
and base.weekday =OV.weekday
left join
avg_rider_attendance as attend
on
base.zone_code=attend.zone_code
and base.hour_of_day_updated=attend.STARTS_AT_LOCAL_HOUR_OF_DAY
and base.weekday=attend.weekday
)
---- total rider count per zone--
, tot_rider_per_zone as
(
select
distinct
zone_code
, sum(total_OV) over (partition by zone_code)/ sum(total_OV) over () as ov_perc
, round (323*sum(total_OV) over (partition by zone_code)/ sum(total_OV) over () )as expected_rider_count
from
final_OP
order by 1
)
-- final output sheet here
, output as
(
select
a.ZONE_CODE as Zone
,a.WEEKDAY as DAY
,a.HOUR_OF_DAY_UPDATED as HOUR
,b.expected_rider_count
,round(a.AVG_CONTRACT_DURATION_WORKED_PRCT *b.expected_rider_count) as Riders_Needed
from final_OP as a
left join
tot_rider_per_zone as b
on a.zone_code=b.zone_code
)
select * from output; -- main table output

"""

conn = snowflake.connector.connect(
        user='Richa.kumari@deliveroo.com',
        password=os.getenv('password'),
        account="deliveroo.eu-central-1",
        authenticator='externalbrowser',
        warehouse="BI_DEVELOPMENT",
        role="BI_DEVELOPMENT",
    )

# Use Snowflake's cursor to execute the parameterized query
cur = conn.cursor()

cur.execute(query)
df = cur.fetch_pandas_all()


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://accounts.google.com/o/saml2/idp?idpid=C01jnk96c&SAMLRequest=nZLBjtowEIZfJXLPiZMsULAIiBJQkegGQZaW3rzOAC6OndrOBvr0NWGRtofdQw%2BRIueb8Tf5Zzg%2Bl8J7AW24kgmKghB5IJkquDwk6Cmf%2B33kGUtlQYWSkKALGDQeDQ0tRUUmtT3KNfyuwVjPNZKGtB8SVGtJFDXcEElLMMQyspl8W5I4CEmllVVMCfSm5OMKagxo6wzvJYXhTu9obUUwbpomaB4CpQ84DsMQhwPsqCvy6c6f3Uzv8BEOO1feEQ5fvbp94fL2Cz7Ser5BhnzN85W%2FyjY58iZ31amSpi5Bb0C%2FcAZP6%2BVNwDiDdLZcbGfrLAug9hlIq6nwo8BI1ewFPQFTZVVb1zpwb3gPBRbqwN30izRB1YkX%2FXMmt%2BXzZhvDbjNfz6J%2B%2Bvl7TrNdvL9kczGgtjqV5%2FTP8ceOIW97jze%2BxrswpoaFvIZq3VEYd%2Fxw4EedPOqQbpd046DXe%2FiJvNSFyiW1beXdnDKmamlNcFDqIKD1U%2FiaYIx5UY3dw4tkGka%2F5GnQY%2Bi2KaS9U4%2F%2Ba%2F4hftvidfMeXRiLdKUEZxdvrnRJ7ftZRUHUnvDC37cogZJyMSkKDca4zIRQzVQDtW7Bra4B4dHt1n9XfPQX&RelayState=51099 to authen

In [4]:
cols=['Zone','Day','Hour','expected_rider_count','Riders Needed']

# Assign new column names
df.columns = cols
df.to_csv('/Users/richakumari/Desktop/Middle East scheduling model -testing codes/inputs_processing/T2_riders_needed.csv')

In [5]:
# df.RIDERS_NEEDED.sum()
# --14277
df.head(1)

,Zone,Day,Hour,expected_rider_count,Riders Needed
0,SMY,7,24,22,3


In [6]:
def remove_nan_from_string(value):
    if pd.isna(value):
        return value  # Return NaN as is if the whole cell is NaN
        # Split the string by comma, filter out 'nan', and join back
    return ','.join([item for item in value.split(',') if item.lower() != 'nan'])


In [7]:

CLUSTERS_PER_MARKET = {
    'Qatar': [
        'Doha'
    ],
    'Dubai': [
        "DBRS-DBS-DEGC-DICP-DJP-DM-EH-HPJLT-IBN-JH-JI-JLT-MDW8-PC-PCG-PCW-PR-SPR-SS-TL-TM",
        "AR-DDH-DHS-DHS2-DIP-DSB-DTS-HPMC-JGE-JVC-JVT-RMRM-SC-TWM",
        "AKW-AMR-AMZ-ANH-AQS-ATW-AWH-DCH-DEIR-DFC-MRDF-WAA",
        "AQ-AQPP-DBB-DBI-DD11-DDO-DIC-DMEY-DSO-DSW-DT-FCOW-HCC-HPDT-JMH-KAR-NAS-RAK-UMS-ZBL"
    ],
    'Abu_Dhabi': [
        "ABN-ABN2-ABS-ADAF-ADAR-ADBS-ADKC-ADMF-ADMFQ-ADMQ-ADMR-ADMS-ADSC-ADSH-ADSK-ADZC-ADZC2-RAHA-SAAD-YAS"
    ],
    'Ajman': [
        "AIA-AJCC-AJFE-AJFW-AJMF-ARBE-ARBW-ARDA-ARYA-AUTY"
    ],
    'Al_Ain': [
        "AAB-AAH-AAJ-AAU-AAY-AMT-AWZ"
    ],
    'Ras_Al_Khaimah': [
        "RKCB-RKDN-RKDS-RKFN-RKGB-RKHE-RKHW-RKMF-RKRE-RKRJ-RKRS-RKRW-RKSQ"
    ],
    'Sharjah': [
        "ARH-AZH-SAJ-SAKZ-SAMJ-SAND-SATN-SAUS-SHAQ-SHUN-SIA-SMW-SMYS-SRQA-SUBSAUSC-SWAT"
    ],
    'Kuwait': [
        'AAM-AAS-AHM-ARD-BYN-DAJ-DSM-FWY-HAW-ISH-JAA-JAB-JAH-JEL-KCE-KCW-KFH-KFZ-KHA-KHR-KSB-KWI-MAH-MAK-MES-MGF-NWS-QAS-QOR-QRW-QSR-RAB-SAM-SBH-SDQ-SHD-SLW-SMY-SUL-SWH-WAM'
        # 'ASA-CAJ-KAJ-KAK-KAZ-KBN-KDY-SAA-WAF'
    ]
}

ZONES_PER_CLUSTER = {
    # Kuwait
    'AAM-AAS-AHM-ARD-BYN-DAJ-DSM-FWY-HAW-ISH-JAA-JAB-JAH-JEL-KCE-KCW-KFH-KFZ-KHA-KHR-KSB-KWI-MAH-MAK-MES-MGF-NWS-QAS-QOR-QRW-QSR-RAB-SAM-SBH-SDQ-SHD-SLW-SMY-SUL-SWH-WAM':
        ['AAM', 'AAS', 'AHM', 'ARD', 'BYN', 'DAJ', 'DSM', 'FWY', 'HAW', 'ISH', 'JAA', 'JAB', 'JAH', 'JEL', 'KCE', 'KCW', 'KFH', 'KHA', 'KSB', 'MAH', 'MAK',
         'MES', 'MGF', 'QAS', 'QOR', 'QSR', 'RAB', 'SBH', 'SDQ', 'SLW', 'SMY', 'SUL', 'SWH', 'WAM','KFZ','KHR','NWS','QRW','SHD'],  # 34
    # 'ASA-CAJ-KAJ-KAK-KAZ-KBN-KDY-SAA-WAF':
    #     ['ASA', 'KAJ', 'KAK', 'KAZ', 'KBN', 'KDY', 'SAA', 'WAF']  # 8
}



In [8]:
def get_market_from_cluster(cluster):
    """
    Returns market in which the cluster is in
    :param cluster: current running cluster
    :return: Market name -> String
    """
    for key in CLUSTERS_PER_MARKET.keys():
        if cluster in CLUSTERS_PER_MARKET[key]:
            return key
            
def prepare_riders_needed_from_consolidated(cluster):
    """
    desc
    :param cluster:
    :return:
    """
    market = get_market_from_cluster(cluster)
    df = cur.fetch_pandas_all()
    
    riders_needed=pd.read_csv('/Users/richakumari/Desktop/Middle East scheduling model -testing codes/inputs_processing/T2_riders_needed.csv')
    # directory_path = '../manual_inputs/gsheets/riders_needed/master'
    
    cluster_zones = ZONES_PER_CLUSTER[cluster]
    riders_needed = riders_needed.loc[riders_needed.Zone.isin(cluster_zones)]
    riders_needed = riders_needed[['Zone',	'Day',	'Hour',	'Riders Needed']]

    riders_needed.to_csv(
            'riders_needed_{0}.csv'.format(cluster), index=False
        )

def prepare_cluster_inputs_from_consolidated(cluster):
    """
    Checks that all inputs are in their place and moves them to the model directory
    :param cluster: current running cluster
    :return: None
    """
    prepare_riders_needed_from_consolidated(cluster)

   

In [9]:
def get_total_available_riders_and_hours_from_master_table(market, hours_per_rider):
    """
    Computes all the available riders that we have to schedule the market
    :return: number of rider hours to schedule
    """
    riders_table = pd.read_csv('/Users/richakumari/Desktop/Middle East scheduling model -testing codes/inputs_processing/KW Rider Ops Fleet Scheduling Matrix - Weekly Scheduling.csv')
    zone_region_map=pd.read_csv('/Users/richakumari/Desktop/Middle East scheduling model -testing codes/inputs_processing/ME Zones - Operating Time - Kuwait.csv')
    
    zone_region_map.columns = [
    'Zone Name',
    'Region',
    'ZONE_CODE',
    'OPEN_HOURS',
    'CLOSE_HOURS',
    'OPEN_HOURS',  
    'CLOSE_HOURS' 
]
    riders_table =  riders_table[riders_table['Agency'].isin(['Thunder_FL', 'Albarq Co_FL'])] # filter for T2 Kuwait riders
   
    riders_table['First Region Pref'] = riders_table['First Pref'].str.split().str[0]
    riders_table['Second Region Pref'] = riders_table['Second Pref'].str.split().str[0]
    riders_table = riders_table.loc[riders_table['Rider ID'] > 0]

    riders_table = riders_table.sample(frac=1).reset_index(drop=True)  # randomise

    # riders_table['total_hours'] = len(riders_table) * hours_per_rider

    # get the zone preferences
    #  ADD THE MISSING ZONES IN THE MAPPING TABLE AND QC THE ZONE X REGION MAPPING
    
    merged_df = pd.merge(riders_table, zone_region_map[['Region','ZONE_CODE']], left_on='First Region Pref', right_on='Region', how='left')
    aggregated_df = merged_df.groupby('First Region Pref')['ZONE_CODE'].apply(lambda x: ','.join(sorted(set(x)))).reset_index()

    riders_table = pd.merge(riders_table, aggregated_df, on='First Region Pref', how='left')
    riders_table.rename(columns={'ZONE_CODE': 'FIRST_PREFERENCE_ZONE_CODES_LIST'}, inplace=True)
    #  join on second preference
    merged_df_second_pref = pd.merge(riders_table, zone_region_map[['Region','ZONE_CODE']], left_on='Second Region Pref', right_on='Region', how='left')
    aggregated_df_second_pref = merged_df_second_pref.groupby('Second Region Pref')['ZONE_CODE'].apply(lambda x: ','.join(sorted(set(x)))).reset_index()
    riders_table = pd.merge(riders_table, aggregated_df_second_pref, on='Second Region Pref', how='left')
    riders_table.rename(columns={'ZONE_CODE': 'SECOND_PREFERENCE_ZONE_CODES_LIST'}, inplace=True)
    
    riders_table.to_csv('riders_table_updated_preference.csv', index=False)

    

    total_rider_hours = len(riders_table) * hours_per_rider

    active_riders = list(riders_table['Rider ID'].unique())
    active_riders = [int(rider) for rider in active_riders]

    return active_riders, total_rider_hours



In [10]:
#  For new riders allocate virtual zone, for old riders keep previous allocated zones
def get_adjusted_shifts_data(active_riders, market='Kuwait'):
    """
    gets shifts from previous week
        (https://deliveroo.looker.com/looks/148382?toggle=fil)
    :return:
    """
    shifts_table = pd.read_csv('/Users/richakumari/Desktop/Middle East scheduling model -testing codes/inputs_processing/past shifts kuwait t2 riders.csv')

    shifts_table = shifts_table.loc[shifts_table['City Name'] == market]

    del shifts_table['Unnamed: 0']
    del shifts_table['City Name']
    del shifts_table['Country Name']

    shifts_table.columns = ['rider_id', 'zone_code', 'starts_at_local', 'slot_id', 'HOD', 'DOW Num']

    shifts_table = shifts_table.loc[shifts_table.rider_id.isin(active_riders)]

    old_riders = shifts_table.rider_id.unique()
    new_riders = [rider for rider in active_riders if rider not in old_riders]
    print('new riders:', len(new_riders))
    for rider in new_riders:
        shifts_table = pd.concat([shifts_table,
                                  pd.DataFrame([[rider, 'virtual_zone', '', 0, 0, '']],
                                               columns=shifts_table.columns)],
                                 ignore_index=True)

    shifts_table['rider_id'] = shifts_table['rider_id'].astype(int)

    return shifts_table


In [11]:

# map the regions preferences to zone and create one column

In [12]:
def get_preferences_ranking_from_preferences_selection(zone, market='Kuwait'):
    market == 'Kuwait' 
    riders_table = pd.read_csv('riders_table_updated_preference.csv')
    
    preferences_table = riders_table.dropna(subset=['FIRST_PREFERENCE_ZONE_CODES_LIST', 'SECOND_PREFERENCE_ZONE_CODES_LIST'])
    preferences_table['FIRST_PREFERENCE_ZONE_CODES_LIST'] = preferences_table['FIRST_PREFERENCE_ZONE_CODES_LIST'].astype(str).str.strip()
    preferences_table['SECOND_PREFERENCE_ZONE_CODES_LIST'] = preferences_table['SECOND_PREFERENCE_ZONE_CODES_LIST'].astype(str).str.strip()

    preferences_table['Preference'] = preferences_table[['FIRST_PREFERENCE_ZONE_CODES_LIST', 'SECOND_PREFERENCE_ZONE_CODES_LIST']].agg(','.join, axis=1)

    def remove_nan_from_string(value):
        if pd.isna(value):
            return value  # Return NaN as is if the whole cell is NaN
        # Split the string by comma, filter out 'nan', and join back
        return ','.join([item for item in value.split(',') if item.lower() != 'nan'])


    preferences_table['Preference'] = preferences_table['Preference'].apply(remove_nan_from_string)
    
    preferences_table['ranking'] = preferences_table['Preference'].apply(lambda x: x.split(','))
    preferences_table['ranking'] = preferences_table['ranking'].apply(lambda x: x.index(zone) if zone in x else None)
    preferences_table['ranking'] = 100 - preferences_table['ranking']
    preferences_table['Rider ID']=preferences_table['Rider ID'].astype(int)
    preferences_table.set_index('Rider ID', inplace=True)

    return preferences_table['ranking']


In [13]:

def compute_eligible_riders(shifts_data, zones, current_rider_hours, hours_per_rider, preferences_feature=True):
    """
    Sets the riders to schedule in the specific set of zones
    :return: list with rider ids
    For every zone X rider combination obtain the ranking for that zone for that rider 
    """
    if zones[0] in [zone for zones_ in [ZONES_PER_CLUSTER[cluster] for cluster in CLUSTERS_PER_MARKET['Kuwait']]
                    for zone in zones_] and preferences_feature:
        
        preferences_ranking = get_preferences_ranking_from_preferences_selection(zones[0])

        preferences_ranking.fillna(-1000, inplace=True)

        shifts_data = shifts_data.merge(preferences_ranking, left_on='rider_id', right_index=True, how='left')

        shifts_data['ranking'].fillna(1, inplace=True)

        all_riders = pd.DataFrame(
            shifts_data.groupby('rider_id').first()['ranking']
        )
        riders_in_zone = all_riders.sort_values('ranking', ascending=False)
        
        #  gives ranking for every rider x zone passed based on preference
        hours_worked_inside_of_zones = pd.DataFrame(
            shifts_data.loc[shifts_data.zone_code.isin(zones)].groupby('rider_id').count()['HOD']
        ).rename(columns={'HOD': 'Hours in Zone'})

        riders_in_zone = riders_in_zone.merge(hours_worked_inside_of_zones, how='left', on='rider_id').fillna(0)
        riders_in_zone['Hours in Zone'] = riders_in_zone['Hours in Zone'] + riders_in_zone['ranking']
        riders_in_zone = riders_in_zone.sort_values('Hours in Zone', ascending=False)
 
    riders_in_zone['Theoric Hours'] = hours_per_rider
    riders_in_zone['Agg Hours'] = riders_in_zone['Theoric Hours'].cumsum()
    
    riders_in_zone = riders_in_zone.loc[riders_in_zone['Agg Hours'] <= current_rider_hours]
    riders_in_zone = riders_in_zone.index.unique()

    return riders_in_zone



In [14]:
# compute_eligible_riders(shifts_table,['MES'],1000 ,8 ,preferences_feature=True)

In [15]:
# shifts_table.head(1)

In [16]:
def get_zone_rider_hours_needed(zone, cluster, cluster_rider_hours):
    """
    Computes the amount of rider hours needed, scaled with the amount we have available
    :param zone: Zone to compute
    :param cluster: Cluster STR
    :param cluster_rider_hours: RH available for the cluster
    :return: scaled amount of rider hours needed
    """

    cluster_riders_needed = pd.read_csv('riders_needed_{0}.csv'.format(cluster))
    cluster_riders_needed_grouped = cluster_riders_needed.groupby('Zone').sum()['Riders Needed']
    pct_of_total = int(cluster_riders_needed_grouped[zone]) / cluster_riders_needed_grouped.sum()

    return pct_of_total * cluster_rider_hours


In [17]:
def compute_cluster_rider_hours_target(total_hours, market='Kuwait'):
    """
    :param total_hours: Total RH to allocate to the specific market
    Each cluster gets proportional RHs depending on the RH Needed of the Cluster
    :return:
    """

    market_hours_per_cluster = []
    market='Kuwait'
    for cluster in CLUSTERS_PER_MARKET[market]:
        cluster_riders_needed = pd.read_csv('riders_needed_{0}.csv'.format(cluster))
        cluster_hours = pd.DataFrame([[cluster, cluster_riders_needed['Riders Needed'].sum()]],
                                     columns=['Cluster', 'Riders Needed'])
        market_hours_per_cluster.append(cluster_hours)
    market_hours_per_cluster = pd.concat(market_hours_per_cluster)

    market_hours_per_cluster['% of RH'] = (market_hours_per_cluster['Riders Needed'] /
                                           market_hours_per_cluster['Riders Needed'].sum())

    market_hours_per_cluster['Rider Hours'] = market_hours_per_cluster['% of RH'] * total_hours

    return market_hours_per_cluster


In [18]:
def compute_riders_zones_table(market, hours_per_rider, preferences_feature=True):
    """
    Computes the rider-zone allocation table based on the riders available and the target R vs R* from local team
    :return: Saves allocation to CSV
    """
    active_riders, total_rider_hours = get_total_available_riders_and_hours_from_master_table(market, hours_per_rider)
    shifts_data_raw = get_adjusted_shifts_data(active_riders, market)

    print('target amount of riders', len(active_riders))
    print('target rider hours', total_rider_hours)

    print('riders in shifts data', len(shifts_data_raw.rider_id.unique()))

    print('current amount of riders:', len(shifts_data_raw.rider_id.unique()))
    print('current rider hours', len(shifts_data_raw))

    for cluster in CLUSTERS_PER_MARKET[market]:
        print(cluster)
        prepare_cluster_inputs_from_consolidated(cluster)

    cluster_rider_hours_target = compute_cluster_rider_hours_target(total_rider_hours, market)

    scheduled_riders = []
    scheduled_zones = []

    market_allocations = []
    zone_tables = []  # Initialize zone_tables here to use it later for zone_riders_assigned calculation

    for cluster in CLUSTERS_PER_MARKET[market]:
        print('-' * 20)
        print(cluster)

        cluster_rider_hours = int(cluster_rider_hours_target.loc[
            cluster_rider_hours_target.Cluster == cluster]['Rider Hours'].iloc[0])

        print('cluster target rider hours:', cluster_rider_hours)

        shifts_data_iter = shifts_data_raw.copy()
        shifts_data_iter = shifts_data_iter.loc[~shifts_data_iter.rider_id.isin(scheduled_riders)]

        cluster_riders = compute_eligible_riders(shifts_data_iter, ZONES_PER_CLUSTER[cluster], cluster_rider_hours,
                                                 hours_per_rider, preferences_feature=True)

        shifts_data = shifts_data_iter.loc[shifts_data_iter.rider_id.isin(cluster_riders)]

        agg_rider_hours = 0
        for zone in ZONES_PER_CLUSTER[cluster]:
            print(zone)

            zone_rider_hours = get_zone_rider_hours_needed(zone, cluster, cluster_rider_hours)

            shifts_data = shifts_data.loc[~shifts_data.rider_id.isin(scheduled_riders)]
            rider_zones = compute_eligible_riders(shifts_data, [zone], zone_rider_hours, hours_per_rider,
                                                  preferences_feature=True)

            agg_rider_hours += len(rider_zones) * hours_per_rider

            scheduled_riders.extend(rider_zones)
            scheduled_zones.append(zone)

            rider_zones_df = pd.DataFrame(columns=['rider_id', 'zone_code'])
            rider_zones_df['rider_id'] = rider_zones
            rider_zones_df['zone_code'] = zone
            zone_tables.append(rider_zones_df)

            print('zone allocated rider hours:', len(rider_zones) * hours_per_rider)

        shifts_data = pd.concat(zone_tables)

        print('cluster allocated rider hours:', len(shifts_data.rider_id.unique()) * hours_per_rider)
        print('cluster allocation error (%)',
              int(100 * ((len(shifts_data.rider_id.unique()) * hours_per_rider) - cluster_rider_hours) / cluster_rider_hours))

        market_allocations.append(shifts_data)
        print('-' * 30)

    # Step 1: Calculate the difference between riders needed and riders assigned for each zone
    zone_riders_needed = pd.read_csv('/Users/richakumari/Desktop/Middle East scheduling model -testing codes/inputs_processing/T2_riders_needed.csv')
    zone_riders_needed = zone_riders_needed.groupby('Zone')['expected_rider_count'].max().reset_index()

    zone_riders_assigned = pd.concat(zone_tables).groupby('zone_code')['rider_id'].count().reset_index()
    zone_riders_assigned.columns = ['Zone', 'Riders Assigned']

    zone_difference = zone_riders_needed.merge(zone_riders_assigned, on='Zone', how='left')
    zone_difference['Riders Assigned'].fillna(0, inplace=True)
    zone_difference['Difference'] = zone_difference['expected_rider_count'] - zone_difference['Riders Assigned']
    zone_ranking = zone_difference.sort_values(by='Difference', ascending=False).reset_index(drop=True)
    zone_ranking['Rank'] = zone_ranking['Difference'].rank(ascending=False, method='first')
    # print(zone_ranking)


    # Step 2: For each rider and the preference zones, pick the top preference based on the above rank
    remaining_riders = [rider for rider in shifts_data_raw.rider_id.unique() if rider not in scheduled_riders]
    print('remaining riders', len(remaining_riders))
    
    # zone_ranking.to_csv('zone_ranking.csv')
    preference_riders_table = pd.read_csv('riders_table_updated_preference.csv')
    
    preferences_table = preference_riders_table.dropna(subset=['FIRST_PREFERENCE_ZONE_CODES_LIST', 'SECOND_PREFERENCE_ZONE_CODES_LIST'])
    preferences_table['FIRST_PREFERENCE_ZONE_CODES_LIST'] = preferences_table['FIRST_PREFERENCE_ZONE_CODES_LIST'].astype(str).str.strip()
    preferences_table['SECOND_PREFERENCE_ZONE_CODES_LIST'] = preferences_table['SECOND_PREFERENCE_ZONE_CODES_LIST'].astype(str).str.strip()
    preferences_table['Preference'] = preferences_table[['FIRST_PREFERENCE_ZONE_CODES_LIST', 'SECOND_PREFERENCE_ZONE_CODES_LIST']].agg(','.join, axis=1)
    preferences_table['Preference'] = preferences_table['Preference'].apply(remove_nan_from_string)
    
    
    def rearrange_zones(row):
        zones = row['Preference'].split(',')
        ranked_zones = sorted(zones, key=lambda x: zone_ranking.set_index('Zone').loc[x, 'Rank'] if x in zone_ranking['Zone'].values else float('inf'))
        return ','.join(ranked_zones)
    
    preferences_table['Rearranged_Preference'] = preferences_table.apply(rearrange_zones, axis=1)
    
    preferences_table['Rearranged_Preference'] = preferences_table['Rearranged_Preference'].str.split(',').str[0]
    # print(preferences_table)


    for rider in remaining_riders:
        preference_zones = preferences_table.loc[preferences_table['Rider ID'] == rider, 'Rearranged_Preference'].iloc[0].split(',')
        top_preference_zone = preference_zones[0]  # Select the first preference zone

        scheduled_riders.append(rider)
        scheduled_zones.append(top_preference_zone)

        rider_zones_df = pd.DataFrame(columns=['rider_id', 'zone_code'])
        rider_zones_df['rider_id'] = [rider]
        rider_zones_df['zone_code'] = [top_preference_zone]
        zone_tables.append(rider_zones_df)

    market_allocations = pd.concat(zone_tables)
    market_allocations.to_csv('/Users/richakumari/Desktop/Middle East scheduling model -testing codes/inputs_processing/riders_table.csv')
    print('total allocated riders:', len(scheduled_riders))
    print('total allocated rider hours:', len(scheduled_riders) * hours_per_rider)
    print('remaining riders:', len(shifts_data_raw.rider_id.unique()) - len(scheduled_riders))
    print(len(set(scheduled_riders)))
    print('scheduled riders:', len(scheduled_riders))
    print('duplicates:', set([x for x in scheduled_riders if scheduled_riders.count(x) > 1]))


In [19]:
compute_riders_zones_table('Kuwait', hours_per_rider=24*7*0.28)  # 8 hrs per day

new riders: 41
target amount of riders 323
target rider hours 15193.920000000002
riders in shifts data 323
current amount of riders: 323
current rider hours 40100
AAM-AAS-AHM-ARD-BYN-DAJ-DSM-FWY-HAW-ISH-JAA-JAB-JAH-JEL-KCE-KCW-KFH-KFZ-KHA-KHR-KSB-KWI-MAH-MAK-MES-MGF-NWS-QAS-QOR-QRW-QSR-RAB-SAM-SBH-SDQ-SHD-SLW-SMY-SUL-SWH-WAM
--------------------
AAM-AAS-AHM-ARD-BYN-DAJ-DSM-FWY-HAW-ISH-JAA-JAB-JAH-JEL-KCE-KCW-KFH-KFZ-KHA-KHR-KSB-KWI-MAH-MAK-MES-MGF-NWS-QAS-QOR-QRW-QSR-RAB-SAM-SBH-SDQ-SHD-SLW-SMY-SUL-SWH-WAM
cluster target rider hours: 15193
AAM
zone allocated rider hours: 141.12
AAS
zone allocated rider hours: 141.12
AHM
zone allocated rider hours: 47.040000000000006
ARD
zone allocated rider hours: 141.12
BYN
zone allocated rider hours: 329.28000000000003
DAJ
zone allocated rider hours: 282.24
DSM
zone allocated rider hours: 188.16000000000003
FWY
zone allocated rider hours: 235.20000000000005
HAW
zone allocated rider hours: 893.7600000000001
ISH
zone allocated rider hours: 188.16000000

In [20]:
#  QC steps, zones -riders needed, riders assigned difference
#  QC step 2, riders preference region and region assigned mismatch check

# QC 

In [21]:

#  QC steps :
# 1. Check difference between riders needed and riders assigned

zone_riders_needed = pd.read_csv('/Users/richakumari/Desktop/Middle East scheduling model -testing codes/inputs_processing/T2_riders_needed.csv')
zone_riders_needed = zone_riders_needed.groupby('Zone')['expected_rider_count'].max().reset_index()

allocation = pd.read_csv('riders_table.csv')
allocation=allocation.groupby('zone_code')['rider_id'].count().reset_index()
allocation.columns = ['Zone', 'Riders Assigned']

zone_difference = zone_riders_needed.merge(allocation, on='Zone', how='left')
zone_difference['Riders Assigned'].fillna(0, inplace=True)
zone_difference['Difference'] = zone_difference['expected_rider_count'] - zone_difference['Riders Assigned']
zone_ranking = zone_difference.sort_values(by='Difference', ascending=False).reset_index(drop=True)
zone_ranking


,Zone,expected_rider_count,Riders Assigned,Difference
0,HAW,22,19.0,3.0
1,FWY,7,5.0,2.0
2,MGF,5,3.0,2.0
3,QSR,18,16.0,2.0
4,MES,17,15.0,2.0
5,KFH,6,4.0,2.0
6,KHA,5,3.0,2.0
7,DSM,5,4.0,1.0
8,SMY,22,21.0,1.0
9,ARD,4,3.0,1.0


In [22]:
# 2. check the region assigned if it is in first or second preference , or none of the above
#  Map the zones assigned to regions
allocation = pd.read_csv('riders_table.csv')
allocation.head(2)

,Unnamed: 0,rider_id,zone_code
0,0,910935,AAM
1,1,912007,AAM


In [23]:
zone_region_map=pd.read_csv('/Users/richakumari/Desktop/Middle East scheduling model -testing codes/inputs_processing/ME Zones - Operating Time - Kuwait.csv')
zone_region_map.columns = [
    'Zone Name',
    'Region',
    'ZONE_CODE',
    'OPEN_HOURS',
    'CLOSE_HOURS',
    'OPEN_HOURS',  
    'CLOSE_HOURS' 
]
zone_region_map
assigned_rider_df = pd.merge(allocation, zone_region_map[['Region','ZONE_CODE']], left_on='zone_code', right_on='ZONE_CODE', how='left')
assigned_rider_df 


,Unnamed: 0,rider_id,zone_code,Region,ZONE_CODE
0,0,910935,AAM,Central,AAM
1,1,912007,AAM,Central,AAM
2,2,869718,AAM,Central,AAM
3,0,688774,AAS,East,AAS
4,1,914941,AAS,East,AAS
...,...,...,...,...,...
318,0,928025,ISH,Central,ISH
319,0,928017,KCE,East,KCE
320,0,928037,ISH,Central,ISH
321,0,928031,KCE,East,KCE


In [24]:
preference_riders_table = pd.read_csv('riders_table_updated_preference.csv')
preference_riders_table['First Region Pref'] = preference_riders_table['First Pref'].str.split().str[0]
preference_riders_table['Second Region Pref'] = preference_riders_table['Second Pref'].str.split().str[0]
# Merge rider_table with rider_details_table on rider_id
merged_table_preference = assigned_rider_df.merge(preference_riders_table[['Rider ID','First Region Pref','Second Region Pref']], left_on='rider_id', right_on ='Rider ID')

# Check if the region present in rider falls in either of the 2 preferences
def check_region_preference(row):
    if row['Region'] not in [row['First Region Pref'], row['Second Region Pref']]:
        return 'Flag'
    else:
        return 'No Flag'

# Apply the check_region_preference function to each row
merged_table_preference['Flag'] = merged_table_preference.apply(check_region_preference, axis=1)

merged_table_preference[merged_table_preference['Flag']=='Flag']


,Unnamed: 0,rider_id,zone_code,Region,ZONE_CODE,Rider ID,First Region Pref,Second Region Pref,Flag
